In [1]:
import numpy as np
import os
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.preprocessing import MinMaxScaler, RobustScaler, scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import pandas as pd
import modules.feature_extraction as fe
import HelperFunctions as hf

#### Features aus csv lesen

In [2]:
df = pd.read_csv(r"data\features.csv", sep=';', header=None)
df = df.iloc[1:, 3:]
df = df.reindex(columns=[4, 5, 6, 7, 3])
df = df.apply(pd.to_numeric, errors='coerce')
df = df[df.iloc[:, -1] < 3]
df = df.sample(frac=1)

#### x y Werte zum Trainieren trennen

In [3]:
x = df.iloc[:, :-1]
y = df.iloc[:, -1]

#### Train / Test Daten trennen

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4)

#### Support Vector Machine trainieren

In [5]:
model = SVC(verbose=1, C=1000, gamma=0.002)
model.fit(x_train, y_train)

[LibSVM]

SVC(C=1000, gamma=0.002, verbose=1)

#### Mit Testdaten vergleichen

In [6]:
y_pred = model.predict(x_test)

#### Accuracy-, Precision-, Recall- und F1-Score ausgeben

In [7]:
print(hf.get_scores(y_test, y_pred))

Accuracy:	0.6127819548872181
Precision:	0.5987371703067192
Recall:		0.6127819548872181
F1-Score:	0.599558972027057


#### Testdaten von Fr. Burghart hier

In [8]:
df = hf.extract_features(os.path.join("data", "images", "test"))

Class names and indices: {'bottleOpener': 0, 'canOpener': 1, 'corcScrew': 2, 'multiTool': 3}


In [9]:
#df = df.iloc[1:, 3:]
df = df.reindex(columns=["Aspect Ratio", "Number of Corners (Harris)",
           "Number of Corners (Shi-Tomasi)",
           "Perimeter Area Ratio", "Class Index"])
df = df.apply(pd.to_numeric, errors='coerce')
df = df[df.iloc[:, -1] < 3]
df = df.sample(frac=1)

In [10]:
x_test = df.iloc[:, :-1]
y_test = df.iloc[:, -1]

y_pred = model.predict(x_test)

print(hf.get_scores(y_test, y_pred))

Accuracy:	0.6428571428571429
Precision:	0.7519841269841271
Recall:		0.6428571428571429
F1-Score:	0.6074829931972789


In [11]:
y_pred = y_pred.astype(int)
class_names = ['Bottle opener', 'Can opener', 'Cork screw', 'Multitool']

for idx, pred in enumerate(y_pred):
    print (f"Bild Nr. {idx}: Tatsächlich: {class_names[y_test[idx]]} | Prediction: {class_names[pred]}")

Bild Nr. 0: Tatsächlich: Bottle opener | Prediction: Bottle opener
Bild Nr. 1: Tatsächlich: Bottle opener | Prediction: Bottle opener
Bild Nr. 2: Tatsächlich: Bottle opener | Prediction: Bottle opener
Bild Nr. 3: Tatsächlich: Bottle opener | Prediction: Bottle opener
Bild Nr. 4: Tatsächlich: Bottle opener | Prediction: Can opener
Bild Nr. 5: Tatsächlich: Can opener | Prediction: Bottle opener
Bild Nr. 6: Tatsächlich: Can opener | Prediction: Bottle opener
Bild Nr. 7: Tatsächlich: Can opener | Prediction: Cork screw
Bild Nr. 8: Tatsächlich: Can opener | Prediction: Can opener
Bild Nr. 9: Tatsächlich: Can opener | Prediction: Can opener
Bild Nr. 10: Tatsächlich: Cork screw | Prediction: Bottle opener
Bild Nr. 11: Tatsächlich: Cork screw | Prediction: Bottle opener
Bild Nr. 12: Tatsächlich: Cork screw | Prediction: Can opener
Bild Nr. 13: Tatsächlich: Cork screw | Prediction: Bottle opener
